# GWU STAT 4197/6197

### Week 7 SAS Code Examples: Summarizing Data
* PROC MEANS
* PROC SUMMARY
* PROC FREQ
* PROC SURVEYFREQ
* PROC SURVEYMEANS
* PROC TABULATE

#### (Source: SAS Documentation for Code Explanation)



### PROC MEANS
* produces summary report with descriptive statistics
* can include the VAR statement that identifies the (numeric) analysis variable (or variables whose statistics need to be comouted) and their in the output
* can include classification variables (character or numeric)  whose values define subgroups

In [3]:
*Ex1_proc_summary_proc_means_sum.sas (Part 1);
proc means data=sashelp.heart sum MEAN MEDIAN Q1 P25 maxdec=2; 
class BP_status;
var weight;
run;

In [1]:
proc means data=sashelp.class mean std Q1 Median Q3;
   var weight;
run;

SAS Connection established. Subprocess id is 9896



### PROC MEANS 

* can include NOPRINT option that suppresses all displayed output

* can include multiple OUTPUT statements to create several OUT= data sets. In that case,
    * \_TYPE_ (an automatic numeric variable) is automatically created
         * tracks the level of summarization
    * \_FREQ_ (an automatic numeric variable) is automatically created
         * provides the number of observations at each level of summarization

* can include other PROC MEANS Statistics
    * e.g., N, MIN, N, MIN, MAX, SUM 
    * e.g., median, Q1, Q3, P1, P3

In [5]:
proc means data=sashelp.class mean std Q1 Median Q3 noprint;
   var weight;
   class sex;
    output out=work.summary 
    Mean = Mean std =std
    Q1=Q1 Median=Median Q3=Q3;
run;

proc print data=work.summary noobs;
run;

Sex,_TYPE_,_FREQ_,Mean,std,Q1,Median,Q3
,0,19,100.026,22.7739,84,99.50,112.5
F,1,9,90.111,19.3839,84,90.00,102.5
M,1,10,108.950,22.7272,85,107.25,128.0


### In PROC SUMMARY, you can add
* CLASS statement
* VAR statement
* OUTPUT statement to create OUT= SAS data set

##### Add the AUTONAME option to the output statement for PROC SUMMARY to automatically rename the conflicting variables

In [15]:
*Ex1_proc_summary_proc_means_sum.sas (Part 2);
proc summary data=sashelp.heart;
     class sex BP_Status;
     var _numeric_;
     output out=work.stats 
     mean=/autoname;
run;
proc print data=stats noobs;
run;

Sex,BP_Status,_TYPE_,_FREQ_,AgeCHDdiag_Mean,AgeAtStart_Mean,Height_Mean,Weight_Mean,Diastolic_Mean,Systolic_Mean,MRW_Mean,Smoking_Mean,AgeAtDeath_Mean,Cholesterol_Mean
,,0,5209,63.3030,44.0687,64.8132,153.087,85.3586,136.910,119.958,9.3665,70.5364,227.417
,High,1,2267,63.3867,46.8496,64.7137,161.846,95.8844,155.735,127.237,8.6819,70.9505,236.503
,Normal,1,2143,63.4786,42.6934,65.0058,149.168,79.8292,127.114,116.124,9.8142,70.2864,222.954
,Optimal,1,799,62.0163,39.8673,64.5789,138.720,70.3242,109.767,109.567,10.1087,68.6647,213.498
Female,,2,2873,65.6547,44.0515,62.5726,141.389,84.6464,136.886,120.771,5.4069,71.5670,228.542
Male,,2,2336,61.5737,44.0899,67.5674,167.466,86.2346,136.938,118.957,14.2473,69.6932,226.051
Female,High,3,1186,66.3370,48.2057,62.3237,150.990,96.0590,159.181,129.983,4.0255,73.1993,240.979
Female,Normal,3,1166,65.5075,42.1921,62.7199,137.313,79.6415,126.616,116.701,5.8110,69.9730,223.954
Female,Optimal,3,521,61.2600,38.7562,62.8090,128.590,69.8676,109.119,108.850,7.6397,65.8235,210.373
Male,High,3,1081,60.9776,45.3617,67.3314,173.779,95.6929,151.955,124.219,13.7940,68.9484,231.636


In [4]:
*Ex1_proc_summary_proc_means_sum.sas (Part 3);
 proc summary data=sashelp.prdsale;
     var _numeric_;
     output out=want_summary(drop=_type_ _freq_) 
     sum=/autoname;
   run;
proc print data=want_summary noobs;
format _numeric_ dollar12.;
run;

ACTUAL_Sum,PREDICT_Sum,QUARTER_Sum,YEAR_Sum,MONTH_Sum
"$730,337","$706,295","$3,600","$2,870,640","$17,860,320"


In [12]:
*Ex1_proc_summary_proc_means_sum.sas (Part 4);
proc transpose data=want_summary 
      out=t_ws (rename=(col1=Amount))
      name=Var_Sum;
run;
proc print data=t_ws noobs;
format amount dollar12.;
run;

Var_Sum,_LABEL_,Amount
ACTUAL_Sum,Actual Sales,"$730,337"
PREDICT_Sum,Predicted Sales,"$706,295"
QUARTER_Sum,Quarter,"$3,600"
YEAR_Sum,Year,"$2,870,640"
MONTH_Sum,Month,"$17,860,320"


### Code Explanation

* The CLASS statement accepts one or more classification variables. 

* If a classification variable has a missing value, SAS will eliminate the entire observation from the analysis. 

* With the CLASS statement, you can use various options that include the ORDER= option, DESCENDING option, MISSING option.  See SAS® documentation for details.

* The VAR statement accepts the analysis variable as numeric. 

* The OUTPUT statements accepts the OUT= data set and allows to specify statistics.

* _TYPE_ is an automatic numeric variable, which can be used to help us track the level of summarization.

* The LEVELS options adds to OUT= data set an automatic numeric variable _LEVEL_. This variable contains a sequential counter of rows within a given value of _TYPE_. 

* The WAYS option adds to OUT= data set an automatic numeric variable _WAY_.  Here, this variable indicates a two-way interaction between the SEX and SMOKING_STATUS CLASSIFICATION variables.


In [29]:
*Ex1B_proc_summary_sum.sas;
options nonumber nodate  ps=58 ls=90;
data heart;
 set sashelp.heart;
if status= 'Alive' then death=0;
else death=1;

if status= 'Alive' then survived =1;
else survived =0;
run;
proc summary data=heart;
   class smoking_status;
   var death survived;
   output out=count_data
      sum(death)=death_count
      sum(survived)=survived_count;
run;
proc print data=count_data; run;

Obs,Smoking_Status,_TYPE_,_FREQ_,death_count,survived_count
1,,0,5173,1971,3202
2,Heavy (16-25),1,1046,443,603
3,Light (1-5),1,579,187,392
4,Moderate (6-15),1,576,213,363
5,Non-smoker,1,2501,891,1610
6,Very Heavy (> 25),1,471,237,234


In [30]:
proc summary data=heart;
   class sex smoking_status;
   var weight;
   output out=stats
      mean=meanWEIGHT/ levels ways;
run;
proc print data=stats; run;


Obs,Sex,Smoking_Status,_WAY_,_TYPE_,_LEVEL_,_FREQ_,meanWEIGHT
1,,,0,0,1,5173,153.088
2,,Heavy (16-25),1,1,1,1046,154.763
3,,Light (1-5),1,1,2,579,146.766
4,,Moderate (6-15),1,1,3,576,144.586
5,,Non-smoker,1,1,4,2501,153.742
6,,Very Heavy (> 25),1,1,5,471,164.081
7,Female,,1,2,1,2856,141.422
8,Male,,1,2,2,2317,167.460
9,Female,Heavy (16-25),2,3,1,339,136.086
10,Female,Light (1-5),2,3,2,422,140.610


### The FREQ procedure 

* can be used on categorical or ordinal variables for 

    * simple frequency distributions
    * 2-way cross-tabulations
    * 3-way tabulations 

In [14]:
*Ex2_PROC_FREQ_SAS;
OPTIONS nocenter nonumber nodate ps=58 ls=90;
title1 'One-Way Table';
title2 'No option on the TABLES statement';
proc freq data=sashelp.heart;
tables sex weight_status bp_status;
run;

### The MISSING option in the TABLES statement

* is used to tell SAS to include missing values in percentage calculations.



In [15]:
title1 'One-Way Table';
title2 'MISSING option';
proc freq data=sashelp.heart;
tables weight_status / missing;
run;

In [16]:
title1 'One-Way Table';
title2 'MISSPRINT option in the TABLES statement';
proc freq data=sashelp.heart;
tables weight_status / missing;
run;


In [17]:
title1 'Two-Way Table';
title2 ' ';
proc freq data=sashelp.heart;
tables weight_status*bp_status;
run;

In [19]:
title1 'Two-Way Table';
title2 'WHERE statement';
proc freq data=sashelp.heart;
tables weight_status*bp_status;
where sex='Male';
run;

In [20]:
title1 'Two-Way Table';
title2 'Suppress Percent';
proc freq data=sashelp.heart;
tables weight_status*bp_status/
       norow nocol nopercent;
run;


In [21]:
title1 'Two-Way Table';
title2 'Suppress Percent & Column Percent';
proc freq data=sashelp.heart;
tables weight_status*bp_status/
       norow nocol nopercent;
run;

In [22]:
title1 'Two-Way Table';
title2 'LIST Option';
proc freq data=sashelp.heart;
tables weight_status*bp_status/LIST
       norow nocol nopercent;
run;


In [23]:
title1 'NOPRINT Option with the TABLE Statement';
title2 'Count and Percent';
proc sort data=sashelp.heart
       (where=(weight_status ne ' '))
       out=heart_x;
by weight_status; run;
proc freq data=heart_x;
 by weight_status; 
 tables bp_status/noprint out=heart_out;
run;
proc print data=heart_out; run;



Obs,Weight_Status,BP_Status,COUNT,PERCENT
1,Normal,High,394,26.7663
2,Normal,Normal,704,47.8261
3,Normal,Optimal,374,25.4076
4,Overweight,High,1839,51.8028
5,Overweight,Normal,1340,37.7465
6,Overweight,Optimal,371,10.4507
7,Underweight,High,32,17.6796
8,Underweight,Normal,97,53.5912
9,Underweight,Optimal,52,28.7293


In [24]:

* Create formats, and the example-data;
proc format;
    value weight_fmt
       1 = 'Less than 90 lbs'
       2 = '90-<120 lbs'
       3 = '120-150 lbs';
run;
Data Class;
 SET sashelp.class;
    if weight <90 THEN weight_grp =1 ;
    else if 90<=weight<120 THEN weight_grp = 2;
    else if weight >= 120 THEN weight_grp = 3;
run;
title1 'One-Way Table No ORDER= Options';
title2 ' ';
proc freq data=class; 
 tables weight_grp;
 Format weight_grp weight_fmt.;
run;


weight_grp,Frequency,Percent,CumulativeFrequency,CumulativePercent
Less than 90 lbs,7,36.84,7,36.84
90-<120 lbs,9,47.37,16,84.21
120-150 lbs,3,15.79,19,100.00


In [25]:
title1 'One-Way Table ORDER= Formatted';
title2 'Formatted values appearing in the ascending order';
proc freq data=class ORDER= Formatted; 
 tables weight_grp;
 Format weight_grp weight_fmt.;
run;

weight_grp,Frequency,Percent,CumulativeFrequency,CumulativePercent
120-150 lbs,3,15.79,3,15.79
90-<120 lbs,9,47.37,12,63.16
Less than 90 lbs,7,36.84,19,100.00


In [26]:
title1 'One-Way Table ORDER= FREQ';
title2 'Order of categories based on frquencies'; 
title3 '(the category with the highest freqency appears first)';
proc freq data=class ORDER= FREQ; 
 tables weight_grp;
 Format weight_grp weight_fmt.;
run;
 /*cancel title2 and title3 */
 title2; title3;

weight_grp,Frequency,Percent,CumulativeFrequency,CumulativePercent
90-<120 lbs,9,47.37,9,47.37
Less than 90 lbs,7,36.84,16,84.21
120-150 lbs,3,15.79,19,100.00


In [27]:
*Adapted from SAS-L ;
*Contributed by data_null - 7/19/2016;
options nonumber nodate  ps=58 ls=90;
data Have;
  input Age   Regular_cnt;
datalines;
1    2814
2    2187
26   1976
51   345
52   678
;
proc format;
   value agegrp
      0-4 ='<5 Years'
      25-30 = '25-30 Years'
      51-High = '>50 Years';
   run;
title1 'WEIGHT Statement in PROC FREQ';
proc freq data=Have;
   tables age;
   format age agegrp.;
   weight regular_cnt;
   run;

Age,Frequency,Percent,CumulativeFrequency,CumulativePercent
<5 Years,5001,62.51,5001,62.51
25-30 Years,1976,24.70,6977,87.21
>50 Years,1023,12.79,8000,100.00


In [28]:
*Calculations of relative Risk using PROC FREQ;
options nonumber nodate  ps=58 ls=90;
data have;
infile datalines firstobs=2;
Label cc='Exposed to Pancreas Cancer';
input Smoking_Status $ 1-10 cc $ 12-19 count 21-23;
datalines;
12345678901234567890123
Smokers    Cases     60
Smokers    Controls 100
Nonsmokers Cases     40
Nonsmokers Controls 300
;
title1 'Relative Risk, and Odds Ratio Calculations';
proc freq data=have;
tables smoking_status*cc /nocol nopercent relrisk;
weight count;
run;


### PROC TABULATE 

The TABULATE can be used to 
* display descriptive statistics (e.g., N, SUM, and Mean) in tabular format
* produce tables in up to three dimensions
* report multiple variables one after another hierarchically within each dimension
* label and format the variables as well as the statistics

This procedure is appropriate for summary reports, for example, when the row collapses or summarizes data based on the group or category variables (Zender, 2008).  In contrast, the PROC PRINT is appropriate for detail reports in which every observation in the data is listed.


### The CLASS statement should include 

* categorical variables (with a limited number of categories). 

#### The TABLEe statement 

* can have up to three dimension expressions as well as the table options.  
The order of the dimensions is page, row, and column.  

* In this example, two dimensions are specified – weight_status variable in the row dimension, and the BP_Status in the column dimension.  Options can be added at the end after a ‘/’.  

In this example, no options are added.  This is essentially a cross-table, and the code below requested two statistics, N and ROWPCTN. See SAS Documentation for the detailed list of statistics.


In [31]:
*Ex3_proc_tabulate.sas (Part 1);
options nonumber nodate ls=132 ps=58 ;
PROC TABULATE data=sashelp.heart format=comma7. ;
  TITLE1 'Two Dimensional TABLE';
  TITLE2 'N and Row Percentage';
  CLASS weight_status BP_status;
  TABLE weight_status all, (BP_Status all)*(N rowpctn*f=6.1);
run;


In [32]:
*Ex3_proc_tabulate.sas (Part 2);
PROC TABULATE data=sashelp.heart format=comma7. ;
TITLE1 'Two Dimensional TABLE';
  TITLE2 'Variable Labels Changed and KEYLABEL Statement Added';
  CLASS weight_status BP_Status;
  KEYLABEL N='Total' rowpctn = 'Row %';
  TABLE weight_status='Body Mass Index Category' all, 
        (BP_Status='Blood Pressure Category' all)*(N rowpctn*f=6.1);
run;

In [33]:
*Ex3_proc_tabulate.sas (Part 3);
PROC TABULATE data=sashelp.heart format=comma7. ;
TITLE1 'Three Dimensional TABLE';
  TITLE2 'Mean Weight';
  CLASS weight_status BP_Status sex;
  VAR  weight;
  KEYLABEL N='Total'  mean = 'Mean (lbs)';
  TABLE (sex all), weight_status='Body Mass Index Category' all, 
        (BP_Status='Blood Pressure Category' all)
        *(N weight*mean);
run;


In [34]:
*Ex3_proc_tabulate.sas (Part 4);
PROC TABULATE data=sashelp.heart format=comma7.;
TITLE1 'Concatenated Rows - Two Dimensional TABLES';
  TITLE2 'Mean Weight';
  CLASS weight_status sex bp_status;
  VAR  weight;
  KEYLABEL N='Total'  mean = 'Mean (lbs)';
  TABLE (sex all)*weight_status='Body Mass Index Category' all, 
         (bp_status='Blood Pressure Category' all)
             *(N weight*mean);
  run;

In [35]:
*Ex4_Multilabel_Format.sas (Part 1);
options nodate nonumber;
proc format ;
 value m_agefmt (multilabel notsorted)
              low-34 = '25-34 Years'
              35-44 = '35-44 Years'
              45-54 = '45-54 Years'
              55-64 = '55-64 Years'
              low-49= '25-49 Years'
              50-64 ='50-64 Years';

    value m_agefmt_x (multilabel)
              low-34 = '25-34 Years'
              35-44 = '35-44 Years'
              45-54 = '45-54 Years'
              55-64 = '55-64 Years'
              low-49= '25-49 Years'
              50-64 ='50-64 Years';

proc tabulate data=sashelp.heart;
  class AgeAtStart/mlf preloadfmt order=data;  
  var AgeAtdeath;
  table AgeAtStart all, 
    n*format=5.0 all*(AgeAtdeath)*mean*format=4.1;
  Format AgeAtStart m_agefmt.;
  title1 'Value m_agefmt (multilabel notsorted)';
  title2 ' Class AgeAtStart/mlf preloadfmt order=data';
run;


In [36]:
*Ex4_Multilabel_Format.sas (Part 2);
proc tabulate data=sashelp.heart;
  class AgeAtStart/mlf;  
  var AgeAtdeath;
  table AgeAtStart all, 
    n*format=5.0 all*(AgeAtdeath)*mean*format=4.1;
  Format AgeAtStart m_agefmt_x.;
  title1 'Value m_agefmt (multilabel)';
  title2 'Class AgeAtStart/mlf';
run;

In [1]:
*Ex9_Surveymeans_ODS_RTF.sas (Part 1);
options nodate nonumber nonotes nosource;
ods exclude all;
ods graphics off;
data person_x;
  length age_grp $20;
  do varstr=1 to 5;
    do varpsu=1 to 3;
   do i=1 to 1000;
           flu=scan('0,1', rantbl(0, .97, .3));
           totexp = 400+rannor(123);
           perwtf=10+ranuni(123);
  if mod(_n_, 2) = 0 then do;
    sex='Female';
    age_grp = '18 years or older';
  end;
  else do;
    sex='Male';
    age_grp = '0-17 years';
  end;
        output;
       end; 
    end; 
  end; 
run;
ods exclude none;

SAS Connection established. Subprocess id is 2476



In [2]:
*Ex9_Surveymeans_ODS_RTF.sas (Part 2);
ods graphics off;
proc surveymeans data=person_x nobs mean ;
  stratum varstr;
  cluster varpsu;
  weight perwtf ;
  var TOTEXP;
 domain sex age_grp age_grp*sex('Male');
  ods output Statistics=ALL domain=Male;
run;